# Datensammlung von Nachrichtenseiten per Web Scraping - FAZ

In [53]:
#Installation Bibliotheken

import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import json

In [54]:
#Definition der Parameter für das Web Scraping 

#Suchwörter in Google Search API
search_queries = ["Künstliche Intelligenz", "AI", "Artificial Intelligence", "KI"]

#Lege ein Startdatum für die automatische Datenerfassung fest
last_day_str = "2024-01-01"

#Zeitintervall von x Tagen vor dem Zieldatum
request_days = 3

#Limit für API-Anfragen
api_request_limit = 8

#Pausenzeit des Data Scraping über Beautiful Soup in Sekunden 
scrap_pause = 3

#Bereinigung der Datumsangabe
last_day = datetime.strptime(last_day_str, "%Y-%m-%d")

#Berechnung des letzten Tages
first_day = last_day - timedelta(days=request_days*api_request_limit) - ((timedelta(days=1)*api_request_limit)-timedelta(days=1))

#Ausgabe des Zeitraums
print(f"Time period: {first_day.strftime('%Y-%m-%d')} until {last_day_str} with {api_request_limit} requests")

Time period: 2023-12-01 until 2024-01-01 with 8 requests


In [55]:
#API-Anfrage

#API-Schlüssel
api_key = "???"

#DataFrame für die Ergebnisse erstellen
df_data_all_queries = []

#Berechnung Zeitintervall für jede Schleife
loop_interval = timedelta(days=request_days)

#Schleife bis zur Erreichung der Grenze an API-Anfragen
for i in range(api_request_limit):
    #Berechnen des Start- und Enddatums für die aktuelle Schleife
    end_date = last_day - (loop_interval * i) - (timedelta(days=1) * i) if i > 0 else last_day - (loop_interval * i)
    start_date = end_date - (timedelta(days=request_days))

    #Bereinigung von Datumsangaben
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')
    
    #Schleife über die Suchbegriffe
    for search_query in search_queries:
        #API-Endpunkt und Parameter
        api_endpoint = "https://www.googleapis.com/customsearch/v1"
        api_params = {
            "key": api_key,
            "cx": "???",
            "q": f"{search_query} before:{end_date_str}",
            "sort": "date",
        }

        #API-Aufruf
        response = requests.get(api_endpoint, params=api_params)
        data = response.json()
        
        #Daten im DataFrame speichern
        if "items" in data:
            for item in data["items"]:
                link = item["link"]
                title = item["title"]

                #Extrahieren von HTML-Inhalten aus der Website
                article_response = requests.get(link)
                
                #Pause von x Sekunden
                time.sleep(scrap_pause)
                
                soup = BeautifulSoup(article_response.text, 'html.parser')

                '''Individueller Teil je Quelle'''
                #Extrahieren des Inhalts
                #Extrahieren Sie den gewünschten JSON-Abschnitt 
                json_script_list = soup.find_all('script', {'type': 'application/ld+json'})
                json_data = {}

                for script in json_script_list:
                    try:
                        script_data = json.loads(script.string)
                        if script_data.get('@type') == 'NewsArticle':
                            json_data = script_data
                            break
                    except json.JSONDecodeError:
                        pass

                #Extrahieren des Artikeltextes
                description = json_data.get("description", "")
                article_body = json_data.get("articleBody", "")

                #Schreiben Sie das Ergebnis in die Variable "content"
                content = description + ' ' + article_body

                #Extrahieren des Datums aus dem HTML-Inhalt   
                meta_tag = soup.find('time', {'class': 'atc-MetaTime'})
                article_date = meta_tag.get('datetime', '') if meta_tag else ''

                #Prüfen, ob ein Wert für article_date vorhanden ist
                #<time>-Tag finden und datetime-Attribut extrahieren
                time_tag = soup.find('time', {'datetime': True})
                article_date = time_tag['datetime'] if time_tag else ''

                #Datum formatieren
                if article_date:
                    formatted_date = datetime.strptime(article_date, "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%d")
                else:
                    formatted_date = "1900-01-01"

                #Titel aus dem HTML-Code extrahieren
                article_title = soup.title.text
                
                '''Individueller Teil je Quelle'''
                #Prüfen, ob das Datum mit dem Enddatum übereinstimmt
                if datetime.strptime(start_date_str, "%Y-%m-%d") <= datetime.strptime(formatted_date, "%Y-%m-%d") <= datetime.strptime(end_date_str, "%Y-%m-%d"):
                    df_data_all_queries.append({
                        "Search Query": search_query,
                        "Quelle": "FAZ",
                        "Datum": formatted_date,
                        "Link": link,
                        "Titel": article_title,
                        "Text": content
                    })

#DataFrame für alle Suchbegriffe erstellen
df_all_queries = pd.DataFrame(df_data_all_queries)

#DataFrame anzeigen mit Prüfen, ob der DataFrame leer ist
if df_all_queries.empty:
    print("No entries found!")
else:
    print(df_all_queries)


                Search Query Quelle       Datum  \
0     Künstliche Intelligenz    FAZ  2023-12-30   
1     Künstliche Intelligenz    FAZ  2023-12-29   
2                         AI    FAZ  2023-12-30   
3                         KI    FAZ  2023-12-31   
4                         KI    FAZ  2023-12-29   
..                       ...    ...         ...   
108  Artificial Intelligence    FAZ  2023-12-03   
109                       KI    FAZ  2023-12-04   
110                       KI    FAZ  2023-12-03   
111                       KI    FAZ  2023-12-01   
112                       KI    FAZ  2023-12-01   

                                                  Link  \
0    https://www.faz.net/podcasts/f-a-z-d-economy-p...   
1    https://www.faz.net/aktuell/technik-motor/digi...   
2    https://www.faz.net/podcasts/f-a-z-d-economy-p...   
3    https://www.faz.net/aktuell/gesellschaft/erstm...   
4    https://www.faz.net/aktuell/politik/ausland/tr...   
..                                     

In [56]:
#Neue Daten in einem Hauptdatensatz in CSV speichern

#Prüfen, ob die CSV-Datei bereits existiert
try:
    #Versuche, die vorhandene CSV-Datei zu lesen
    existing_df = pd.read_csv("2_Daten_FAZ_V2.csv")
except FileNotFoundError:
    #Wenn die Datei nicht existiert, erstelle einen leeren DataFrame
    existing_df = pd.DataFrame()

#Anhängen der neuen Daten an den bestehenden DataFrame
df = pd.DataFrame(df_all_queries)
df_to_append = pd.concat([existing_df, df], ignore_index=True)

#Duplikate auf Basis aller Spalten entfernen
df_to_append = df_to_append.drop_duplicates()

#Speichern des kombinierten DataFrame in der CSV-Datei
df_to_append.to_csv("2_Daten_FAZ_V2.csv", index=False)